[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/chatbots/conversational-agents/langchain-lex-agent.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/chatbots/conversational-agents/langchain-lex-agent.ipynb)

# Ask Lex Agent

**Note**: This notebook is not free to run, you will need to create ~20K OpenAI `text-embedding-ada-002` embedding which do cost money. The Pinecone index can be run within the free tier.

In [ ]:
!pip install -qU datasets pod-gpt pinecone-client[grpc] langchain openai tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 KB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.9/472.9 KB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.2/443.2 KB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Set api keys, etc:

In [ ]:
OPENAI_API_KEY = "OPENAI_API_KEY"  # platform.openai.com
PINECONE_API_KEY = "PINECONE_API_KEY"  # app.pinecone.io
PINECONE_ENV = "PINECONE_ENV"

First we download a prebuilt Lex Fridman podcast transcriptions dataset:

In [ ]:
from datasets import load_dataset

data = load_dataset(
    'jamescalam/lex-transcripts',
    split='train'
)
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/jamescalam___json/jamescalam--lex-transcripts-a6ef9b24c11dc3c4/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


Dataset({
    features: ['video_id', 'channel_id', 'title', 'published', 'transcript', 'source'],
    num_rows: 98
})

Initialize the `indexer` object.

In [ ]:
import pod_gpt

indexer = pod_gpt.Indexer(
    openai_api_key=OPENAI_API_KEY,
    pinecone_api_key=PINECONE_API_KEY,
    pinecone_environment=PINECONE_ENV,
    index_name="pod-gpt"
)

In [ ]:
from tqdm.auto import tqdm

for row in data:
    row['published'] = row['published'].strftime('%Y%m%d')
    indexer(pod_gpt.VideoRecord(**row))

In [ ]:
import pinecone

pinecone.init(
    api_key=PINECONE_API_KEY,  # app.pinecone.io
    environment=PINECONE_ENV  # next to API key in console
)

index_name = "pod-gpt"

if index_name not in pinecone.list_indexes():
    raise ValueError(
        f"No '{index_name}' index exists. You must create the index before "
        "running this notebook. Please refer to the walkthrough at "
        "'github.com/pinecone-io/examples'."  # TODO add full link
    )

index = pinecone.Index(index_name)

Initialize the retrieval components (embedding model and vector DB)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectordb = Pinecone(
    index=index,
    embedding_function=embeddings.embed_query,
    text_key="text"
)

Initialize `gpt-3.5-turbo` chat model:

In [ ]:
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name='gpt-3.5-turbo'
)

We then initialize the QA retrieval object using our `llm` and the `vectordb.as_retriever()`:

In [ ]:
from langchain.chains import RetrievalQA

retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

One additional thing we have here is the `chain_type="stuff"`. There are two options here, `"stuff"` or `"map_reduce"`. The `map_reduce` option essentially summarizes returned documents, whereas the `stuff` option just returns the retrieved documents as is.

The `retriever` is ready and can be used by us like this. However, we need to convert it into a `Tool` to be used by our conversational agent. To do that we need the `retriever` itself, a tool description, and a tool name. We use these to initialize the tool like so:

In [ ]:
tool_desc = """Use this tool to answer user questions using Lex
Fridman podcasts. If the user states 'ask Lex' use this tool to get
the answer. This tool can also be used for follow up questions from
the user."""

In [ ]:
from langchain.agents import Tool

tools = [Tool(
    func=retriever.run,
    description=tool_desc,
    name='Lex Fridman DB'
)]

With that, we're ready to initialize the conversational agent. As it is a *conversational* agent, it does need some form of [conversational memory](https://www.pinecone.io/learn/langchain-conversational-memory/). For this we will use the `ConversationBufferWindowMemory` option, which will *remember* the previous `k` interactions between the user and the AI.

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",  # important to align with agent prompt (below)
    k=5,
    return_messages=True
)

In [ ]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent='chat-conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=2,
    early_stopping_method="generate",
    memory=memory,
)

Important items in `agent` parameter:

* `chat-conversational`: for chatbots with conversational memory.
* `react`: refers to the ReAct framework.
* `description`: because the LLM relies on the tool description to decide which tool to use.

### Conversational Agent Prompt

The prompt of the conversational agent is fairly complex. Let's create it then break it down.

In [ ]:
conversational_agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, A

In [ ]:
sys_msg = """You are a helpful chatbot that answers the user's questions.
"""

prompt = conversational_agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
conversational_agent.agent.llm_chain.prompt = prompt

We can see the prompt template like so:

In [ ]:
conversational_agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Lex Fridman DB: Use this tool to answer user questions using Lex\nFridman podcasts. If the user states \'ask Lex\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTION

The conversational agent prompt is defined by the `ChatPromptTemplate`. Let's break it down:

In [ ]:
conversational_agent.agent.llm_chain.prompt.input_variables

['input', 'chat_history', 'agent_scratchpad']

 This prompt template contains *three* `input_variables`, those are:

* `input`: The new user input to the chatbot, i.e. our prompt/query.

* `chat_history`: We defined this above in the `ConversationBufferWindowMemory` definition.

* `agent_scratchpad`: This is where we store the thoughts of the LLM as it is deciding which tools to interact with and *how* to interact with them.

These `input_variables` are fed into the `messages` contained within the prompt template, let's see what we have there:

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Lex Fridman DB: Use this tool to answer user questions using Lex\nFridman podcasts. If the user states \'ask Lex\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me please, please output a response in one of two formats:\n\n**Option 1:**\n


It's a little hard to see here, but there are **three** components in `messages`. Those are:

* `SystemMessagePromptTemplate`

* `MessagesPlaceholder`

* `HumanMessagePromptTemplate`

Let's start with the first item, the `SystemMessage`:

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={})

In [ ]:
print(
    conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template
)

You are a helpful chatbot that answers the user's questions.



That is our initial system message that we set earlier with the `sys_msg`. There's not much to say about this other than it is used to "prime" (set the initial objective of) the model.

Next we have the `MessagesPlaceholder`:

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[1]

MessagesPlaceholder(variable_name='chat_history')

We can see from `'chat_history'` (this must align to the `memory_key` from the `ConversationBufferWindowMemory` initialized earlier) that this is where the previous messages of the conversation will be fed into the LLM.

The format of this input is set by the type of conversational memory being used, which in this case is the `ConversationBufferWindowMemory`.

Finally, we have the `HumanMessagePromptTemplate`:

In [ ]:
conversational_agent.agent.llm_chain.prompt.messages[2]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Lex Fridman DB: Use this tool to answer user questions using Lex\nFridman podcasts. If the user states \'ask Lex\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me please, please output a response in one of two formats:\n\n**Option 1:**\nUse this if you want the human to use a tool.\nMarkdown code snippet formatted in the following schema:\n\n```json\n{{\n    "action": string \\ The action to take. Must be one of Lex Fridman DB\n    "action_input": string \\ The input to the action\n}}\n```\n\n**Option #2:**\nUse this if you want to respond directly

In [ ]:
print(
    conversational_agent.agent.llm_chain.prompt.messages[2].prompt.template
)

TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

> Lex Fridman DB: Use this tool to answer user questions using Lex
Fridman podcasts. If the user states 'ask Lex' use this tool to get
the answer. This tool can also be used for follow up questions from
the user.

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me please, please output a response in one of two formats:

**Option 1:**
Use this if you want the human to use a tool.
Markdown code snippet formatted in the following schema:

```json
{{
    "action": string \ The action to take. Must be one of Lex Fridman DB
    "action_input": string \ The input to the action
}}
```

**Option #2:**
Use this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:

```json
{{
    "action": "Final Answer",
    "action_input": string \ You should put 

This is the most interesting component. First, we have a single `input` — the user's query/prompt. But before this we see a lot of text, the majority of this is the setup for the LLM to be able to use any tools that we've passed to the conversational agent.

In our case, there is just one tool, the `Lex Fridman DB` tool that we defined earlier. We can also see the tool description that we defined. The LLM will use this tool description to figure out which tool (if any) it should use.

## Having a Conversation

Let's begin our conversation. We'll start as any typical conversation begins:

In [ ]:
conversational_agent("hi how are you")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!"
}

> Finished chain.


{'input': 'hi how are you',
 'chat_history': [HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={})],
 'output': "I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!"}

Looks good. We should note that there is this **AgentExecutor chain** thing. Where we can see an `"action"` and an `"action_input"`. It is here where the agent is deciding whether it should use a tool.

Here we see the agent decides on `"action": "Final Answer"`, meaning no tool is required. Therefore, it just uses the LLM as per usual to generate an answer. That answer can be seen in `"I'm just a chatbot, I don't have feelings, but thanks for asking! How can I assist you today?"`.

What if we mention the words `"ask lex"`?

In [ ]:
conversational_agent("ask lex about the future of ai")



> Entering new AgentExecutor chain...
{
    "action": "Lex Fridman DB",
    "action_input": "What did Lex Fridman say about the future of AI?"
}
Observation: Lex Fridman discussed the potential of AI to increase the quality of life, cure diseases, increase material wealth, and help people be happier and more fulfilled. He also mentioned the need for AI to be aligned with humans and not hurt or limit them. Additionally, he talked about the possibility of AI systems exhibiting human-like emotions and feelings, which would require difficult discussions about how to treat them and what role they would have in society. Finally, he speculated about the future of humans and robots in a multi-planetary species world, hoping for a one-to-one ratio and emphasizing the importance of humans being part of the equation.
Thought:{
    "action": "Final Answer",
    "action_input": "Lex Fridman discussed the potential of AI to increase the quality of life, cure diseases, increase material wealth, and

{'input': 'ask lex about the future of ai',
 'chat_history': [HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={}),
  HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={})],
 'output': 'Lex Fridman discussed the potential of AI to increase the quality of life, cure diseases, increase material wealth, and help people be happier and more fulfilled. He also mentioned the need for AI to be aligned with humans and not hurt or limit them. Additionally, he talked about the possibility of AI systems exhibiting human-like emotions and feelings, which would require difficult discussions about how to treat them and what role they would have in society. Finally, he speculated about the future of 

In [ ]:
conversational_agent("what does he think about space exploration?")



> Entering new AgentExecutor chain...
{
    "action": "Lex Fridman DB",
    "action_input": "What does Lex Fridman think about space exploration?"
}
Observation: Lex Fridman is very enthusiastic about space exploration and believes that it is one of the most inspiring things humans can do. He thinks that reaching out into the unknown inspires people on Earth and can also inspire a new generation of scientists. He also believes that the current explosion of new capabilities in the space industry is driving incredible innovation and making it a great time to be in space.
Thought:{
    "action": "Final Answer",
    "action_input": "Lex Fridman is very enthusiastic about space exploration and believes that it is one of the most inspiring things humans can do. He thinks that reaching out into the unknown inspires people on Earth and can also inspire a new generation of scientists. He also believes that the current explosion of new capabilities in the space industry is driving incredible i

{'input': 'what does he think about space exploration?',
 'chat_history': [HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={}),
  HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={}),
  HumanMessage(content='ask lex about the future of ai', additional_kwargs={}),
  AIMessage(content='Lex Fridman discussed the potential of AI to increase the quality of life, cure diseases, increase material wealth, and help people be happier and more fulfilled. He also mentioned the need for AI to be aligned with humans and not hurt or limit them. Additionally, he talked about the possibility of AI systems exhibiting human-like emotions and feelings, which would require difficult discussions about ho

In [ ]:
conversational_agent("ask lex about the future of AI, include the title and url of this information if possible")



> Entering new AgentExecutor chain...
```json
{
    "action": "Lex Fridman DB",
    "action_input": "What is the future of AI? Please provide the title and URL of the source if possible."
}
```
Observation: The context provided includes multiple conversations about the future of AI, but there is no single source or article that provides a definitive answer to this question. The opinions and speculations about the future of AI vary widely depending on who you ask and what their expertise is. Some people are optimistic about the potential benefits of AI for humanity, while others are concerned about the risks and potential dangers. If you have a specific aspect of the future of AI that you are interested in, I can try to provide more information based on the context provided.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "There is no single source or article that provides a definitive answer to the question of the future of AI. The opinions and speculations about 

{'input': 'ask lex about the future of AI, include the title and url of this information if possible',
 'chat_history': [HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={}),
  HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={}),
  HumanMessage(content='ask lex about the future of ai', additional_kwargs={}),
  AIMessage(content='Lex Fridman discussed the potential of AI to increase the quality of life, cure diseases, increase material wealth, and help people be happier and more fulfilled. He also mentioned the need for AI to be aligned with humans and not hurt or limit them. Additionally, he talked about the possibility of AI systems exhibiting human-like emotions and feelings, whic

Great, we can see that the first thing the agent did was default to the `"Lex Fridman DB"` tool. The input to that tool was generated by the LLM, and is `"What did Lex Fridman say about the future of AI?"`.

This input is then passed into the `Lex Fridman DB` tool and the output observation of the LLM (after it has read all of the information returned by our vector DB is returned to our agent. From this observation the agent moves on to the `"Final Answer"` action, giving us the output.